## This panel will provide a bam file viewer

In [1]:
import os, sys, io, random, subprocess, time
import string
import numpy as np
import pandas as pd
pd.set_option('display.width', 200)
from importlib import reload

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment
from Bio import AlignIO, SeqIO
from pyfaidx import Fasta

from IPython.display import HTML

from bokeh.plotting import figure
from bokeh.models import (ColumnDataSource, Plot, LinearAxis, Grid, Range1d,CustomJS, Slider,
                          HoverTool, NumeralTickFormatter, Label, Range1d)
from bokeh.models.glyphs import Text, Rect
from bokeh.layouts import gridplot, column
import panel as pn
import panel.widgets as pnw
pn.extension()

from pybioviz import dashboards, plotters, utils


In [2]:
reload(utils)
reload(plotters)
cov = utils.get_coverage('SRR650219.sorted.bam','RD900MAF',1260,1560)
#print (cov)
p = plotters.plot_coverage(cov)
pn.pane.Bokeh(p)

31


Bokeh(Figure)

In [4]:
#reference = 'Mbovis-AF212297.2.gb'
#filename = 'wt_mbovis.bam'
#chrom = 'NC_002945.4'
#gff = 'Mbovis_AF212297.gff'

filename = 'SRR650219.sorted.bam'
chrom = 'RD900MAF'
gff = 'RD900MAF.gff'

In [5]:
def get_chrom(bam_file):
    """Get first sequence name in a bam file"""
    
    import pysam
    samfile = pysam.AlignmentFile(filename, "r")
    iter=samfile.fetch(start=0,end=10)
    for read in iter:
        if read.reference_name:
            return read.reference_name

get_chrom(filename)

'RD900MAF'

In [6]:
reload(utils)
reload(plotters)
out = pn.pane.Bokeh()

#feats = utils.genbank_to_features('Mbovis-AF212297.2.gb')
feats = utils.gff_to_features('Mbovis_AF212297.gff')

out.object = plotters.plot_features(feats, plot_width=900,plot_height=150)
out

Bokeh(Figure)

In [6]:
out = pn.pane.Bokeh()

def get_fasta_names(filename):
    """Get names of fasta sequences"""

    refseq = Fasta(filename)
    return list(refseq.keys())

def get_fasta_sequence(filename, start, end, key=0):
    """Get chunk of indexed fasta sequence at start/end points"""
    
    from pyfaidx import Fasta
    refseq = Fasta(filename)
    if type(key) is int:
        chrom = list(refseq.keys())[key]
    print (chrom)
    seq = refseq[chrom][start:end].seq
    return seq

#fasta_file='genomes/cow/ARS-UCD1.2/ars-ucd1.2.fa'
fasta_file ='Mbovis-AF212297.fa'
#print (get_fasta_names(fasta_file))
seq = get_fasta_sequence(fasta_file, 1400, 1850, key=0)
print (seq)
out.object = plotters.plot_sequence(seq, tools='xpan,xwheel_zoom')
out

ENA|BX248333|BX248333.1
CCGTGATCACACAACCGTCATGTACGCCCAACGCAAGATCCTGTCCGAGATGGCCGAGCGCCGTGAGGTCTTTGATCACGTCAAAGAACTCACCACTCGCATCCGTCAGCGCTCCAAGCGCTAGCACGGCGTGTTCTTCCGACAACGTTCTTAAAAAAACTTCTCTCTCCCAGGTCACACCAGTCACAGAGATTGGCTGTGAGTGTCGCTGTGCACAAACCGCGCACAGACTCATACAGTCCCGGCGGTTCCGTTCACAACCCACGCCTCATCCCCACCGACCCAACACACACCCCACAGTCATCGCCACCGTCATCCACAACTCCGACCGACGTCGACCTGCACCAAGACCAGACTGTCCCCAAACTGCACACCCTCTAATACTGTTACCGAGATTTCTTCGTCGTTTGTTCTTGGAAAGACAGCGCTGGGGATCGTTCGCTGGATACC


Bokeh(Figure)

In [9]:
def three_frame_translation(seq):
    
    return protseqs


In [7]:
def get_fasta_length(filename):
    """Get length of reference sequence"""

    refseq = Fasta(filename)
    key = list(refseq.keys())[0]
    l = len(refseq[key])
    return l

get_fasta_length('Mbovis-AF212297.fa')


4345492

In [10]:
def plot_bam_alignment(bam_file, chr, start, end, height=50, fontsize="12pt", 
                       plot_width=800, plot_height=250, fill_color='gray'):
    """Bokeh bam alignments plotter.
    Args:
        bam_file: name of a sorted bam file
        start: start of range to show
        end: end of range        
    """
    
    if bam_file is None:
        return plotters.plot_empty('no bam file',plot_width=plot_width, plot_height=plot_height)
    
    h=.6+height/plot_height
    #get outer ranges to retrieve reads from so that we
    #cover the visible range from start-end
    o = (end-start)/2
    #get reads in range into a dataframe
    df = utils.get_bam_aln(bam_file, chr, start-o, end+o)
    if df is None:
        p = plotters.plot_empty('no bam file or bam not indexed')
        return p

    if len(df)==0:
        p = plotters.plot_empty('no data in range')
        return p
    #offset for rects
    df['x'] = df.start+df.length/2
    #set colors by quality
    df['color'] = df.apply(lambda x: 'red' if x.mapq==0 else fill_color ,1)
    df['span'] = df.apply(lambda x: str(x.start)+':'+str(x.end),1)
    
    source = ColumnDataSource(df)
    x_range=(start, end)
    hover = HoverTool(
        tooltips=[            
            ("name", "@name"),
            ("cigar", "@cigar"),
            ("span", "@span"),
            ("length", "@length"),
            ("mapq", "@mapq"),
            ("counts", "@counts")
        ],
        point_policy='follow_mouse',
    )
    tools=[hover,"ypan","save"]
    
    p = figure(title=None, plot_width=plot_width, plot_height=plot_height, x_range=x_range, y_range=(0,height), tools=tools, 
                    min_border=0, toolbar_location='right')
    rects = Rect(x="x", y="y", width="length", height=h, fill_color="color", line_color='gray',fill_alpha=0.4)
    #if start-end > 100:
    p.ygrid.visible = False
    p.add_glyph(source, rects)
    p.yaxis.visible = False
    p.xaxis.formatter = NumeralTickFormatter(format="(0,0)")
    p.toolbar.logo = None
    return p

In [ ]:
reload(utils)
bam_file='wt_mbovis.bam'
#bam_file='SRR650219.sorted.bam'
bam_file = '/storage/nexusmap_rnaseq/mapped_star/2152_M9_Aligned.sortedByCoord.out.bam'
refname = utils.get_chrom(bam_file)
print(refname)

vals=[200000,1221300]
plots = []
for s in vals:
    e=s+3000
    st=time.time()
    #df = utils.get_bam_aln(bam_file, refname, s, e)
    p = plot_bam_alignment(bam_file,refname, s, e, plot_width=480, plot_height=240, height=100)
    print (time.time()-st)
    #cov = utils.get_coverage(bam_file,refname,s,e)    
    plots.append(pn.pane.Bokeh(p))

p1,p2 = plots
pn.Row(p1,p2)

In [11]:
reload(utils)
reload(plotters)

def bam_viewer(bam_file, ref_file, gff_file=None, width=1000, height=200, color='gray'):
    """Bam viewer widget.
    
    Args:
        bam_file: sorted bam file
        ref_file: reference sequence in fasta format
        gff_file: optional genomic features file
    """
    slider = pnw.IntSlider(name='location',start=1,end=10000,value=500,step=500,width=300)
    main_pane = pn.pane.Bokeh(height=100)
    cov_pane = pn.pane.Bokeh(height=60)
    loc_pane = pn.pane.Str(50,width=250,style={'margin': '4pt'})
    feat_pane = pn.pane.Bokeh(height=60)
    ref_pane = pn.pane.Bokeh(height=60)
    xzoom_slider = pnw.IntSlider(name='x zoom',start=50,end=8000,value=1000,step=10,width=100)
    yzoom_slider = pnw.IntSlider(name='y zoom',start=10,end=100,value=20,step=5,width=100)#,orientation='vertical')
    panleft_btn = pnw.Button(name='<',width=50,button_type='primary')
    panright_btn = pnw.Button(name='>',width=50,button_type='primary')
    chroms_select = pnw.Select(name='Chromosome', options=[], width=250)
    colorby = pnw.Select(name='Color by', options=['quality','pair orientation','read strand'], width=180)
    search_pane = pnw.TextInput(name='search',width=200)
    trans_option = pnw.Checkbox(name='show translation')
    debug_pane = pn.pane.Markdown()
    
    def pan_right(event):
        plot = main_pane.object
        plot.x_range
        start = slider.value 
        loc = slider.value+100    
        slider.value=loc
        update(event)
        return

    def pan_left(event):
        loc = slider.value-100
        if loc<1:
            return
        slider.value=loc
        update(event)
        return

    def update_features():
        """Load features"""

        if gff_file is None:
            return        
        ext = os.path.splitext(gff_file)[1]        
        if ext in ['.gff','.gff3']:
            feats = utils.gff_to_features(gff_file)
        elif ext in ['.gb','.gbff']:
            feats = utils.genbank_to_features(gff_file)
        p = feat_pane.object = plotters.plot_features(feats, 
                                                  plot_width=width, plot_height=100)
        return p

    def search_features(event):
        """Find a feature"""
        
        term = search_pane.value        
        feats = utils.gff_to_features(gff_file)
        df = utils.features_to_dataframe(feats)    
        df['gene'] = df.gene.fillna('')
        f = df[df.gene.str.contains(term)].iloc[0]
        debug_pane.object = str(f.start)
        slider.value = int(f.start)
        update(event)
        return
    
    def update_ref(filename, start, end):
        """Update reference sequence"""

        if filename == None:
            return
        seqlen = utils.get_fasta_length(filename)
        slider.end = seqlen
        refseq = Fasta(filename)
        chroms = list(refseq.keys())
        chroms_select.options = chroms
        key = chroms[0]
        seq = refseq[key][int(start):int(end)].seq
        ref_pane.object = plotters.plot_sequence(seq, plot_height=50,fontsize='9pt',xaxis=False)
        return
    
    def update(event):
        """Update viewers on widget change"""

        xzoom = xzoom_slider.value
        yzoom = yzoom_slider.value
        start = slider.value     
        N = xzoom/2
        end = start+N
        chrom = utils.get_chrom(bam_file)
        loc_pane.object = '%s:%s-%s' %(chrom,start,int(end))
        cov = utils.get_coverage(bam_file,chrom,start,end)
        cov_pane.object = plotters.plot_coverage(cov,plot_width=width)
        main_pane.object = plot_bam_alignment(bam_file,chrom,start,end,height=yzoom,plot_width=width,plot_height=height)        
        update_ref(ref_file, start, end)
        if feature_plot:
            feature_plot.x_range.start = start
            feature_plot.x_range.end = end
        debug_pane.object = ''
        return

    slider.param.watch(update, 'value')
    xzoom_slider.param.watch(update, 'value')
    yzoom_slider.param.watch(update, 'value')
    panright_btn.param.watch(pan_right, 'clicks')
    panleft_btn.param.watch(pan_left, 'clicks')
    search_pane.param.watch(search_features, 'value')
    feature_plot = update_features()
    
    #initialise slider
    slider.param.trigger('value')
    
    #menus = pn.Row(bam_input, ref_input, gff_input)
    top = pn.Row(slider,xzoom_slider,yzoom_slider,panleft_btn,panright_btn,loc_pane)
    bottom = pn.Row(chroms_select, search_pane,colorby,trans_option)
    app = pn.Column(top,cov_pane,main_pane,ref_pane,feat_pane,bottom,debug_pane)
    return app

bam_viewer('wt_mbovis.bam', 'Mbovis-AF212297.fa', 'Mbovis_AF212297.gff', width=1000)


163


Column
    [0] Row
        [0] IntSlider(end=4345492, name='location', start=1, step=500, value=500, width=300)
        [1] IntSlider(end=8000, name='x zoom', start=50, step=10, value=1000, width=100)
        [2] IntSlider(end=100, name='y zoom', start=10, step=5, value=20, width=100)
        [3] Button(button_type='primary', name='<', width=50)
        [4] Button(button_type='primary', name='>', width=50)
        [5] Str(str, style={'margin': '4pt'}, width=250)
    [1] Bokeh(Figure, height=60)
    [2] Bokeh(Figure, height=100)
    [3] Bokeh(Figure, height=60)
    [4] Bokeh(Figure, height=60)
    [5] Row
        [0] Select(name='Chromosome', options=['ENA|BX248333|BX248333.1'...], value='ENA|BX248333|BX248333.1', width=250)
        [1] TextInput(name='search', width=200)
        [2] Select(name='Color by', options=['quality', 'pair orientat...], value='quality', width=180)
        [3] Checkbox(name='show translation')
    [6] Markdown(str)

In [ ]:
reload(utils)

template = utils.get_template()
tmpl = pn.Template(template)
tmpl.add_panel('A',main)
#tmpl

In [ ]:
#bam_view('/storage/nexusmap_rnaseq/mapped_star/2152_M9_Aligned.sortedByCoord.out.bam', 
#         '/storage/genomes/cow/bos_taurus_alt.fa', 
#         '/storage/genomes/cow/UMD_3.1.1_renamed.gff3', width=1000)